# Training the Change Detection Model

In [1]:
import os
import re
import cv2
import glob
import json
import time
import random
import string
import imageio
import collections
import numpy as np
from PIL import Image
from typing import Tuple
from shapely.geometry import Polygon
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from datetime import datetime
from models.changedetection_model import ChangeDetectionModel

# Import dataset that relies on synthetic_anno.json for change detection annotations
from dataset.syntheticpairs_dataset import SyntheticPairsDataSet

In [2]:
model_save_path = ""

In [3]:
annotation_path = "C:\\Users\\Ugne\\Documents\\studies\\Python\\DLGroupTask\\synthetic_anno.json"
json_file = open(annotation_path)
anno = json.load(json_file)
json_file.close()

In [5]:
available_scenes = [im["scene"] for im in anno["images"]]
available_scenes_df = pd.DataFrame(available_scenes, columns=["scene_id"])

len(available_scenes)

500

## Split into training, test, validation sets

In [9]:
total_scenes = len(available_scenes)
random.seed(123)
train_scenes = random.sample(available_scenes, int(total_scenes*0.8))
test_scenes = random.sample([s for s in available_scenes if s not in train_scenes], int(total_scenes*0.1))
validation_scenes = [s for s in available_scenes if s not in train_scenes+test_scenes]

In [10]:
def save_scenes_as_txt(scenes, split):
    df = pd.DataFrame(scenes, columns=["scene_id"])
    df.to_csv(f"imagesets\\{split}.txt", header=False, index=False)

In [11]:
#save_scenes_as_txt(train_scenes, "train")
#save_scenes_as_txt(test_scenes, "test")
#save_scenes_as_txt(validation_scenes, "validation")

## Training

In [12]:
# This dictionary defines hyperparameters for the dataset
# Complete configurations for each experiment can be found in the configs folder
train_set_params = {
        "dataset_name": "syntheticpairs",
        "root": "D:\\DeepLearningFiles\\renders_multicam_diff_all",
        "loader_params": {
            "batch_size": 8,
            "drop_last": False,
            "pin_memory": True,
            "num_workers": 8
        },
        "mode": "train",
        "crop": True,
        "resize": True,
        "spatial_resolution": [256, 180],
        "overfit": False,
        "normalize": True,
        "augment": True,
        "shuffle": True
    }

trainset = SyntheticPairsDataSet(train_set_params)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_set_params["loader_params"]["batch_size"],
                                          shuffle=train_set_params["shuffle"], num_workers=train_set_params["loader_params"]["num_workers"])



In [13]:
# Print number of batches
len(trainloader)

50

In [14]:
model_params = dict({
        "model_name": "changedetection",
        "num_classes": 4,
        "dataset_name": "syntheticpairs",
        "rgb": True, # Use RGB inputs
        "depth": False, # Use depth inputs
        "spatial_resolution": [256, 180],
        "mode": "train",
        "max_epochs": 50, # originally set to 4000
        "lr": 0.0004,
        "save_path": "/app/saved_models/changedetection_benchmark/",
        "weights_path": "/app/saved_models/changedetection_benchmark/",
        "load_weights": -1,
        "val_epochs": [1950],
        "val_rate": 50,
        "save_rate": 50,
        "lr_policy": "cosine",
        "lr_decay_iters": 10,
        "batch_size": 16,
        "loss_weights": True
    })
model = ChangeDetectionModel(model_params).model

c:\Users\Ugne\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Ugne\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loaded weights into backbone
Number total params  39643962


In [9]:
# Print model
model

SingleStream(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

In [15]:
# Parallelize model across GPUs if available
if torch.cuda.is_available():
    model = model.cuda()
    model = torch.nn.DataParallel(model)

# Loss definitions
loss_weights = torch.Tensor([0.0035252888617000786, 0.33161259399877635, 0.3316452266650099, 0.3332168904745137]).cuda()
criterion_loss = torch.nn.CrossEntropyLoss(weight=loss_weights) # Segmentation output loss

# Optimizer definitions
optimizer = torch.optim.SGD(model.parameters(), lr=model_params["lr"])

In [16]:
# Main training loop over epochs
starting_epoch = 0
num_epochs = model_params['max_epochs']
for epoch in range(starting_epoch, num_epochs):
    train_iterations = len(trainloader)
    train_batch_size = model_params['batch_size']
    
    model.train()
    running_loss = 0
    # Training loop over samples within epoch
    for i, data in enumerate(trainloader):
        # Process data to correct types
        img1, img2, label, scene, depth1, depth2 = data
        img1 = img1.float()
        img2 = img2.float()
        depth1 = depth1.float()
        depth2 = depth2.float()
        label = label.type(torch.LongTensor)
        
        # Move data to GPU if available
        if torch.cuda.is_available():
            img1 = img1.cuda()
            img2 = img2.cuda()
            depth1 = depth1.cuda()
            depth2 = depth2.cuda()
            label = label.cuda()
        
        # Get model output
        model_input = (img1, img2, depth1, depth2)
        model_out = model(model_input)
        
        # Calculate loss function, get gradients, and update network weights
        loss = criterion_loss(model_out, label)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()

    print("Epoch {} Loss: ".format(epoch), running_loss/len(trainloader))

Epoch 0 Loss:  1.3090244793891908
Epoch 1 Loss:  1.2153567337989808
Epoch 2 Loss:  1.172402606010437
Epoch 3 Loss:  1.1575059366226197
Epoch 4 Loss:  1.1177576315402984
Epoch 5 Loss:  1.0860601687431335
Epoch 6 Loss:  1.0983562302589416
Epoch 7 Loss:  1.0451079034805297
Epoch 8 Loss:  1.027583167552948
Epoch 9 Loss:  1.0147731256484986
Epoch 10 Loss:  1.0061317873001099
Epoch 11 Loss:  1.001140410900116
Epoch 12 Loss:  0.9811407697200775
Epoch 13 Loss:  0.9747023952007293
Epoch 14 Loss:  0.9448469614982605
Epoch 15 Loss:  0.9424292063713073
Epoch 16 Loss:  0.9260956430435181
Epoch 17 Loss:  0.9206092834472657
Epoch 18 Loss:  0.8931335294246674
Epoch 19 Loss:  0.8853948831558227
Epoch 20 Loss:  0.9065267217159271
Epoch 21 Loss:  0.9099127554893494
Epoch 22 Loss:  0.893166835308075
Epoch 23 Loss:  0.8657380020618439
Epoch 24 Loss:  0.8511970496177673
Epoch 25 Loss:  0.8275364065170288
Epoch 26 Loss:  0.8512530553340912
Epoch 27 Loss:  0.8257962715625763
Epoch 28 Loss:  0.8405906248092652

In [17]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S").replace(":","").replace(" ","").replace("-","")

torch.save(model.state_dict(), f'change_detection_model_{current_time}.pt')